## 1. Importar Bibliotecas

In [1]:
import pandas as pd
import json
from collections import Counter
import requests
import csv
import os
import re
import asyncio
from playwright.async_api import async_playwright
import nest_asyncio

# Permitir event loops aninhados (necessário para Jupyter + Playwright)
nest_asyncio.apply()

## 2. Coleta de Dados da API DataJud

In [2]:
url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjce/_search"
api_key = "APIKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw=="

payload = json.dumps({
    "size": 10000,
    "query": {"match": {"assuntos.codigo": "12487"}}, # Código do assunto "Medicamentos"
    "sort": [{"dataAjuizamento": {"order": "desc"}}]
})

headers = {
    'Authorization': api_key,
    'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)
dados_dict = response.json()
print(f"Total de processos encontrados: {dados_dict['hits']['total']['value']}")

# Salvando em Json
with open("dados_completos.json", "w", encoding="utf-8") as f:
    json.dump(dados_dict, f, indent=2, ensure_ascii=False)
print("Dados completos salvos em 'dados_completos.json'")

Total de processos encontrados: 6046
Dados completos salvos em 'dados_completos.json'
Dados completos salvos em 'dados_completos.json'


## 3. Processamento dos Dados (DataFrame)

In [3]:
# Extrair dados dos processos
processos = []

for hit in dados_dict['hits']['hits']:
    processo = hit['_source']
    numero_processo = processo['numeroProcesso']
    grau = processo['grau']
    classe = processo['classe']['nome']
    assuntos = processo['assuntos']
    data_ajuizamento = processo['dataAjuizamento']
    ultima_atualizacao = processo['dataHoraUltimaAtualizacao']
    formato = processo['formato']['nome']
    codigo = processo['orgaoJulgador']['codigo']
    orgao_julgador = processo['orgaoJulgador']['nome']
    municipio = processo['orgaoJulgador']['codigoMunicipioIBGE']
    
    try:
        movimentos = processo['movimentos']
    except:
        movimentos = []
    
    processos.append([numero_processo, classe, data_ajuizamento, ultima_atualizacao, formato,
                     codigo, orgao_julgador, municipio, grau, assuntos, movimentos])

df = pd.DataFrame(processos, columns=['numero_processo', 'classe', 'data_ajuizamento', 'ultima_atualizacao',
                                      'formato', 'codigo', 'orgao_julgador', 'municipio', 'grau', 'assuntos', 'movimentos'])

print(f"Total de processos no DataFrame: {len(df)}")
df.sample(5)

Total de processos no DataFrame: 6046


,numero_processo,classe,data_ajuizamento,ultima_atualizacao,formato,codigo,orgao_julgador,municipio,grau,assuntos,movimentos
5185,06339957820218060000,Agravo de Instrumento,20210922000000,2024-08-20T19:42:40.151Z,Eletrônico,88700,GADES - MARIA REGINA OLIVEIRA CAMARA,2304400,G2,"[{'codigo': 12487, 'nome': 'Fornecimento de me...","[{'codigo': 92, 'nome': 'Publicação', 'dataHor..."
4828,02002860720228060090,Cumprimento Provisório de Sentença,20220223000000,2024-08-20T15:38:25.382Z,Eletrônico,84100,2ª VARA CIVEL DA COMARCA DE ICO,2304400,G1,"[{'codigo': 12487, 'nome': 'Fornecimento de me...","[{'complementosTabelados': [{'codigo': 2, 'val..."
59,02098577020258060001,Apelação Cível,20250813000000,2025-10-13T12:00:14.632Z,Eletrônico,84268,GADES - JORIZA MAGALHAES PINHEIRO,2304400,G2,"[{'codigo': 12485, 'nome': 'Fornecimento de in...","[{'complementosTabelados': [{'codigo': 4, 'val..."
1235,00135387120248060064,Procedimento Comum Cível,20240608000000,2025-03-08T20:33:14.304Z,Eletrônico,14334,VARA UNICA DA INFANCIA E JUVENTUDE DA COMARCA ...,2303709,G1,"[{'codigo': 12487, 'nome': 'Fornecimento de me...","[{'codigo': 1061, 'nome': 'Disponibilização no..."
765,02707705220248060001,Procedimento Comum Cível,20240924000000,2025-09-10T20:25:25.200Z,Eletrônico,8505,23ª VARA CIVEL DA COMARCA DE FORTALEZA,2304400,G1,"[{'codigo': 12487, 'nome': 'Fornecimento de me...","[{'codigo': 92, 'nome': 'Publicação', 'dataHor..."


## 4. Análise de Decisões
#### Identificar processos com decisões de Procedência ou Improcedência

In [4]:
# Extrair decisões dos movimentos
decisoes_por_processo = []
tipos_decisao_contagem = []

for idx, row in df.iterrows():
    numero = row['numero_processo']
    movimentos = row['movimentos']
    grau = row['grau']
    
    decisoes_encontradas = []
    
    if movimentos:
        for mov in movimentos:
            nome_mov = mov.get('nome', '')
            
            termos_decisao = [
                "Procedência",
                "Improcedência",
                "Improcedência do pedido e improcedência do pedido contraposto"
            ]
            
            if (any(palavra in nome_mov for palavra in termos_decisao) and grau == 'G1'):
                decisoes_encontradas.append(nome_mov)
                tipos_decisao_contagem.append(nome_mov)
    
    if decisoes_encontradas:
        decisoes_por_processo.append({
            'numero_processo': numero,
            'decisoes': decisoes_encontradas
        })

print(f"Processos de instância G1 com decisões: {len(decisoes_por_processo)} de {len(df)}")
print(f"\nTipos de decisões encontradas:")
for tipo, count in Counter(tipos_decisao_contagem).most_common(10):
    print(f"  {tipo}: {count}")

Processos de instância G1 com decisões: 1799 de 6046

Tipos de decisões encontradas:
  Procedência: 1284
  Procedência em Parte: 259
  Improcedência: 222
  Procedência em parte do pedido e improcedência do pedido contraposto: 30
  Improcedência do pedido e improcedência do pedido contraposto: 14
  Procedência do pedido e improcedência do pedido contraposto: 5
  Procedência do Pedido - Reconhecimento pelo réu: 3
  Procedência em parte do pedido e procedência do pedido contraposto: 1
  Procedência do pedido e procedência do pedido contraposto: 1


In [5]:
# Criar DataFrame de decisões
decisoes_lista = []

for item in decisoes_por_processo:
    for decisao in item['decisoes']:
        decisoes_lista.append({
            'numero_processo': item['numero_processo'],
            'tipo_decisao': decisao
        })

df_decisoes = pd.DataFrame(decisoes_lista)
df_decisoes.head(10)

,numero_processo,tipo_decisao
0,30039390220258060071,Improcedência
1,02187331420258060001,Procedência
2,30465508320258060001,Procedência em Parte
3,30048329320258060167,Procedência
4,30415587920258060001,Procedência em Parte
5,02162085920258060001,Procedência
6,30355412720258060001,Procedência
7,30022788520258060071,Procedência
8,30303751420258060001,Procedência em Parte
9,30292622520258060001,Procedência


## 5. Seleção Estratificada de Registros
#### Balancear dataset: Todos os "Improcedência" + Amostra estratificada de "Procedência"

In [6]:
# Separar decisões por tipo
df_procedencia = df_decisoes[df_decisoes['tipo_decisao'] == 'Procedência'].copy()
df_improcedencia = df_decisoes[df_decisoes['tipo_decisao'] == 'Improcedência'].copy()
df_improcedencia_contraposto = df_decisoes[df_decisoes['tipo_decisao'] == 'Improcedência do pedido e improcedência do pedido contraposto'].copy()

# Registros de Improcedência
df_improcedencias_todos = pd.concat([df_improcedencia, df_improcedencia_contraposto])

# Seleção estratificada de Procedência
procedencias = []
tamanho_bloco = 20
pulo = 50
posicao = 0
total_procedencia = len(df_procedencia)

while len(procedencias) < 370 and posicao < total_procedencia:
    fim_bloco = posicao + tamanho_bloco
    procedencias.extend(range(posicao, fim_bloco))
    posicao = fim_bloco + pulo
    
procedencias = procedencias[:370]

# Selecionar os registros de Procedência
df_procedencia_selecionada = df_procedencia.iloc[procedencias].copy()

# Combinar todos os registros selecionados
df_decisoes_balanceado = pd.concat([df_procedencia_selecionada, df_improcedencias_todos])

print(f"\nTotal de registros no dataset balanceado: {len(df_decisoes_balanceado)}")
print(f"  - Procedência: {len(df_procedencia_selecionada)}")
print(f"  - Improcedência: {len(df_improcedencia)}")
print(f"  - Improcedência contraposto: {len(df_improcedencia_contraposto)}")

df_decisoes_balanceado.head(10)


Total de registros no dataset balanceado: 606
  - Procedência: 370
  - Improcedência: 222
  - Improcedência contraposto: 14


,numero_processo,tipo_decisao
1,02187331420258060001,Procedência
3,30048329320258060167,Procedência
5,02162085920258060001,Procedência
6,30355412720258060001,Procedência
7,30022788520258060071,Procedência
9,30292622520258060001,Procedência
13,30012724320258060071,Procedência
14,30193154420258060001,Procedência
15,30186884020258060001,Procedência
16,30182752720258060001,Procedência


## 6. Exportar Resultados para Web Scraping

In [7]:
# Salvar DataFrames em CSV
df.to_csv('processos_completo.csv', sep=',', header=True, index=False)
df_decisoes_balanceado.to_csv('decisoes_resumo.csv', sep=',', header=True, index=False)

# Salvar números de processos balanceados em CSV
numeros_balanceados = df_decisoes_balanceado['numero_processo'].unique()
with open('numeros_processos.csv', 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['numeroProcesso'])
    for num in numeros_balanceados:
        writer.writerow([num])

# Salvar dados completos em JSON
with open('processos_com_decisoes.json', 'w', encoding='utf-8') as f:
    json.dump(decisoes_por_processo, f, indent=2, ensure_ascii=False)

print("Arquivos exportados:")
print("  - processos_completo.csv")
print("  - decisoes_resumo.csv")
print(f"  - numeros_processos.csv ({len(numeros_balanceados)} processos únicos)")
print("  - processos_com_decisoes.json")

Arquivos exportados:
  - processos_completo.csv
  - decisoes_resumo.csv
  - numeros_processos.csv (599 processos únicos)
  - processos_com_decisoes.json


## 7. Web Scraping - TJCE
### Funções auxiliares para coleta de dados complementares

In [8]:
# Constantes para Web Scraping
CACHE_FILE = "cache_processos.json"
PALAVRAS_INVALIDAS_JUIZ = ["Especial", "Cível", "Criminal", "Direito", "Vara"]
PADROES_JUIZ = [
    r"Juiz(?:a)?\s+de\s+Direito\s*[:\-]\s*([A-ZÀÁÂÃÇÉÊÍÓÔÕÚ][a-zàáâãçéêíóôõú]+(?:\s+[A-ZÀÁÂÃÇÉÊÍÓÔÕÚ][a-zàáâãçéêíóôõú]+)+)",
    r"Juiz(?:a)?\s*[:\-]\s*([A-ZÀÁÂÃÇÉÊÍÓÔÕÚ][a-zàáâãçéêíóôõú]+(?:\s+[A-ZÀÁÂÃÇÉÊÍÓÔÕÚ][a-zàáâãçéêíóôõú]+)+)",
    r"Juiz(?:a)?[^>]{0,80}?([A-ZÀÁÂÃÇÉÊÍÓÔÕÚ][a-zàáâãçéêíóôõú]{2,}(?:\s+(?:de|da|do|dos|das)\s+[A-ZÀÁÂÃÇÉÊÍÓÔÕÚ][a-zàáâãçéêíóôõú]+|\s+[A-ZÀÁÂÃÇÉÊÍÓÔÕÚ][a-zàáâãçéêíóôõú]{2,})+)",
]
TIPOS_PARTE = ["Requerente", "Autor", "Massa Falida"]

def validar_nome_juiz(nome):
    """Valida se o nome do juiz é válido"""
    if not nome:
        return False
    if re.match(r"^de\s+", nome, re.IGNORECASE):
        return False
    if nome.endswith((" de", " da", " do")):
        return False
    if any(palavra in nome for palavra in PALAVRAS_INVALIDAS_JUIZ):
        return False
    return True

def carregar_cache():
    """Carrega resultados do cache se existir"""
    if os.path.exists(CACHE_FILE):
        try:
            with open(CACHE_FILE, "r", encoding="utf-8") as f:
                cache = json.load(f)
                print(f"✓ Cache carregado: {len(cache)} processos já coletados")
                return cache
        except Exception as e:
            print(f"⚠ Erro ao carregar cache: {e}")
    return []

def salvar_cache(resultados):
    """Salva resultados no cache"""
    try:
        with open(CACHE_FILE, "w", encoding="utf-8") as f:
            json.dump(resultados, f, indent=2, ensure_ascii=False)
    except Exception as e:
        print(f"⚠ Erro ao salvar cache: {e}")

print("Funções auxiliares carregadas!")

Funções auxiliares carregadas!


In [9]:
async def buscar_dados_processo(page, numero_processo):
    """
    Busca os dados do processo no site do TJCE
    Retorna: dict com numero_processo, juiz, requerente, status
    """
    try:
        # Navegar para a página inicial
        await page.goto("https://esaj.tjce.jus.br/cpopg/open.do", wait_until="domcontentloaded")
        await page.wait_for_load_state("networkidle")

        # Selecionar opção "Outros"
        radio_outros = page.get_by_role("radio", name="Outros")
        await radio_outros.check()
        await asyncio.sleep(0.5)

        # Preencher o campo de busca
        campo_busca = page.get_by_role("textbox", name="Número do processo")
        await campo_busca.click()
        await campo_busca.fill(numero_processo)

        # Clicar no botão Consultar
        botao_consultar = page.get_by_role("button", name="Consultar")
        await botao_consultar.click()

        # Aguardar carregamento
        await page.wait_for_load_state("networkidle", timeout=15000)

        # Verificar se processo não existe
        try:
            mensagem_erro = page.get_by_text("Não existem informações")
            if await mensagem_erro.is_visible(timeout=2000):
                print("   Processo não encontrado")
                return {
                    "numero_processo": numero_processo,
                    "juiz": None,
                    "requerente": None,
                    "status": "nao_encontrado"
                }
        except Exception:
            pass

        # Extrair nome do juiz
        juiz = None
        try:
            juiz_element = page.locator("#juizPrimeiraDivTable span").first
            if await juiz_element.is_visible(timeout=2000):
                juiz = (await juiz_element.inner_text()).strip()
        except Exception:
            pass

        # Se não encontrou na tabela, buscar usando regex
        if not juiz:
            try:
                conteudo_html = await page.content()
                for padrao in PADROES_JUIZ:
                    match = re.search(padrao, conteudo_html, re.IGNORECASE)
                    if match:
                        nome_candidato = match.group(1).strip()
                        if validar_nome_juiz(nome_candidato):
                            juiz = nome_candidato
                            break
            except Exception:
                pass

        # Extrair nome do requerente
        requerente = None
        for tipo_parte in TIPOS_PARTE:
            try:
                elemento = (
                    page.locator("#tablePartesPrincipais")
                    .locator(f'td:has-text("{tipo_parte}")')
                    .locator("xpath=following-sibling::td[1]")
                )
                if await elemento.is_visible(timeout=2000):
                    texto_completo = await elemento.inner_text()
                    requerente = texto_completo.split("\n")[0].strip()
                    break
            except Exception:
                continue

        # Determinar status
        status = "sucesso" if (juiz and requerente) else "dados_incompletos"

        print(f"   Juiz = {juiz}\n   Requerente = {requerente}")

        return {
            "numero_processo": numero_processo,
            "juiz": juiz,
            "requerente": requerente,
            "status": status
        }

    except Exception as e:
        print(f"   Erro: {str(e)}")
        return {
            "numero_processo": numero_processo,
            "juiz": None,
            "requerente": None,
            "status": "erro"
        }

print("Função de scraping carregada!")

Função de scraping carregada!


In [1]:
import atexit
import asyncio

async def executar_scraping():
    """Executa o processo de scraping"""
    print("=" * 60)
    print("SCRAPER TJCE - Coleta de Dados de Processos")
    print("=" * 60)

    # Ler números dos processos
    print("\n1. Lendo números dos processos...")
    numeros_processos = []
    with open("numeros_processos.csv", "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            numeros_processos.append(row["numeroProcesso"])
    print(f"   Total de processos a buscar: {len(numeros_processos)}")

    # Carregar decisões
    print("\n2. Carregando decisões...")
    decisoes_map = {}
    try:
        with open("decisoes_resumo.csv", "r", encoding="utf-8") as f:
            reader = csv.DictReader(f)
            for row in reader:
                numero = row["numero_processo"]
                tipo = row["tipo_decisao"]
                sentenca_favoravel = "Procedência" in tipo
                decisoes_map[numero] = sentenca_favoravel
        print(f"   ✓ Decisões carregadas: {len(decisoes_map)} processos")
    except Exception as e:
        print(f"   ⚠ Erro ao carregar decisões: {e}")

    # Carregar cache
    print("\n3. Verificando cache...")
    resultados = carregar_cache()
    processos_ja_coletados = {r["numero_processo"] for r in resultados}
    processos_pendentes = [num for num in numeros_processos if num not in processos_ja_coletados]

    if not processos_pendentes:
        print("   ✓ Todos os processos já foram coletados!")
        return resultados, decisoes_map

    print(f"   Processos pendentes: {len(processos_pendentes)}")

    # Iniciar scraping
    print("\n4. Iniciando coleta de dados...")

    async with async_playwright() as playwright:
        browser = await playwright.chromium.launch(headless=False)
        context = await browser.new_context()
        page = await context.new_page()

        processos_no_cache = len(resultados)
        for idx, numero in enumerate(processos_pendentes, 1):
            total_geral = processos_no_cache + idx
            print(f"\n[{total_geral}/{len(numeros_processos)}] Processando {numero}...")

            resultado = await buscar_dados_processo(page, numero)
            resultados.append(resultado)

            # Salvar cache a cada 50 processos
            if idx % 50 == 0:
                salvar_cache(resultados)

            await asyncio.sleep(1)

            async def shutdown_playwright ():
                await context.close()
                await browser.close()
                await playwright.stop()

            atexit.register(lambda: asyncio.run(shutdown_playwright()))

    salvar_cache(resultados)
    return resultados, decisoes_map

    print("Função carregada!")

### Executar Web Scraping
⚠️ **Atenção**: Esta célula pode levar vários minutos para ser executada

In [ ]:
import atexit
import asyncio

async def executar_scraping():
    """Executa o processo de scraping"""
    print("=" * 60)
    print("SCRAPER TJCE - Coleta de Dados de Processos")
    print("=" * 60)

    # Ler números dos processos
    print("\n1. Lendo números dos processos...")
    numeros_processos = []
    with open("numeros_processos.csv", "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            numeros_processos.append(row["numeroProcesso"])
    print(f"   Total de processos a buscar: {len(numeros_processos)}")

    # Carregar decisões
    print("\n2. Carregando decisões...")
    decisoes_map = {}
    try:
        with open("decisoes_resumo.csv", "r", encoding="utf-8") as f:
            reader = csv.DictReader(f)
            for row in reader:
                numero = row["numero_processo"]
                tipo = row["tipo_decisao"]
                sentenca_favoravel = "Procedência" in tipo
                decisoes_map[numero] = sentenca_favoravel
        print(f"   ✓ Decisões carregadas: {len(decisoes_map)} processos")
    except Exception as e:
        print(f"   ⚠ Erro ao carregar decisões: {e}")

    # Carregar cache
    print("\n3. Verificando cache...")
    resultados = carregar_cache()
    processos_ja_coletados = {r["numero_processo"] for r in resultados}
    processos_pendentes = [num for num in numeros_processos if num not in processos_ja_coletados]

    if not processos_pendentes:
        print("   ✓ Todos os processos já foram coletados!")
        return resultados, decisoes_map

    print(f"   Processos pendentes: {len(processos_pendentes)}")

    # Iniciar scraping
    print("\n4. Iniciando coleta de dados...")

    async with async_playwright() as playwright:
        browser = await playwright.chromium.launch(headless=False)
        context = await browser.new_context()
        page = await context.new_page()

        processos_no_cache = len(resultados)
        for idx, numero in enumerate(processos_pendentes, 1):
            total_geral = processos_no_cache + idx
            print(f"\n[{total_geral}/{len(numeros_processos)}] Processando {numero}...")

            resultado = await buscar_dados_processo(page, numero)
            resultados.append(resultado)

            # Salvar cache a cada 50 processos
            if idx % 50 == 0:
                salvar_cache(resultados)

            await asyncio.sleep(1)

            async def shutdown_playwright ():
                await context.close()
                await browser.close()
                await playwright.stop()

            atexit.register(lambda: asyncio.run(shutdown_playwright()))

    salvar_cache(resultados)
    return resultados, decisoes_map

# Executar scraping
resultados_scraping, decisoes_map = await executar_scraping()

# Estatísticas
print("\n" + "=" * 60)
print("ESTATÍSTICAS")
print("=" * 60)
print(f"Total de processos: {len(resultados_scraping)}")
print(f"Sucesso: {sum(1 for r in resultados_scraping if r['status'] == 'sucesso')}")
print(f"Não encontrados: {sum(1 for r in resultados_scraping if r['status'] == 'nao_encontrado')}")
print(f"Dados incompletos: {sum(1 for r in resultados_scraping if r['status'] == 'dados_incompletos')}")
print(f"Erros: {sum(1 for r in resultados_scraping if r['status'] == 'erro')}")

SCRAPER TJCE - Coleta de Dados de Processos

1. Lendo números dos processos...
   Total de processos a buscar: 599

2. Carregando decisões...
   ✓ Decisões carregadas: 599 processos

3. Verificando cache...
   Processos pendentes: 599

4. Iniciando coleta de dados...


NotImplementedError: 

## 8. Salvar Resultados do Scraping

In [ ]:
# Filtrar apenas processos válidos
resultados_filtrados = [
    r for r in resultados_scraping 
    if r["status"] not in ["nao_encontrado", "erro"]
]

# Adicionar id e sentenca_favoravel
resultados_completos = [
    {
        "id": idx,
        "numero_processo": r["numero_processo"],
        "juiz": r.get("juiz"),
        "requerente": r.get("requerente"),
        "sentenca_favoravel": decisoes_map.get(r["numero_processo"]),
        "status": r["status"]
    }
    for idx, r in enumerate(resultados_filtrados, 1)
]

# Salvar em JSON
with open("dados_processos_tjce.json", "w", encoding="utf-8") as f:
    json.dump(resultados_completos, f, indent=2, ensure_ascii=False)

# Salvar em CSV
with open("dados_processos_tjce.csv", "w", encoding="utf-8", newline="") as f:
    writer = csv.DictWriter(
        f,
        fieldnames=["id", "numero_processo", "juiz", "requerente", "sentenca_favoravel", "status"]
    )
    writer.writeheader()
    writer.writerows(resultados_completos)

print(f"✓ Arquivos salvos ({len(resultados_completos)} processos válidos):")
print("  - dados_processos_tjce.json")
print("  - dados_processos_tjce.csv")

## 9. Inferência de Sexo
### Carregar base de dados de nomes brasileiros

In [ ]:
# Configurações
ARQUIVO_NOMES = "nomes.csv.gz"

def extrair_primeiro_nome(nome_completo):
    """Extrai o primeiro nome de um nome completo"""
    if pd.isna(nome_completo) or nome_completo == "":
        return None
    primeiro_nome = str(nome_completo).strip().split()[0]
    return primeiro_nome.upper()

def buscar_sexo(primeiro_nome, df_nomes, coluna_nome="nome", coluna_sexo="sexo"):
    """Busca o sexo no banco de dados de nomes"""
    if primeiro_nome is None:
        return "Indefinido"
    
    resultado = df_nomes[df_nomes[coluna_nome].str.upper() == primeiro_nome]
    
    if len(resultado) == 0:
        return "Indefinido"
    elif len(resultado) == 1:
        return resultado.iloc[0][coluna_sexo]
    else:
        sexo_mais_comum = resultado[coluna_sexo].mode()
        return sexo_mais_comum[0] if len(sexo_mais_comum) > 0 else "Indefinido"

print("Funções de inferência carregadas!")

In [ ]:
# Carregar banco de nomes
print(f"Carregando banco de nomes de {ARQUIVO_NOMES}...")
df_nomes = pd.read_csv(
    ARQUIVO_NOMES,
    compression="gzip",
    encoding="utf-8",
    on_bad_lines="skip"
)

print(f"Total de nomes carregados: {len(df_nomes)}")
print(f"Colunas disponíveis: {df_nomes.columns.tolist()}")
df_nomes.head()

### Aplicar inferência de sexo nos dados coletados

In [ ]:
# Carregar dados dos processos
df_processos = pd.read_csv("dados_processos_tjce.csv", encoding="utf-8")

print(f"Total de processos: {len(df_processos)}")
print("\nInferindo sexo do juiz e requerente...")

# Ajuste as colunas conforme seu arquivo de nomes
coluna_nome = "first_name"  # AJUSTE AQUI SE NECESSÁRIO
coluna_sexo = "classification"  # AJUSTE AQUI SE NECESSÁRIO

# Extrai primeiro nome e infere sexo
df_processos["primeiro_nome_juiz"] = df_processos["juiz"].apply(extrair_primeiro_nome)
df_processos["primeiro_nome_requerente"] = df_processos["requerente"].apply(extrair_primeiro_nome)

df_processos["sexo_juiz"] = df_processos["primeiro_nome_juiz"].apply(
    lambda x: buscar_sexo(x, df_nomes, coluna_nome, coluna_sexo)
)

df_processos["sexo_requerente"] = df_processos["primeiro_nome_requerente"].apply(
    lambda x: buscar_sexo(x, df_nomes, coluna_nome, coluna_sexo)
)

# Remove colunas temporárias
df_processos = df_processos.drop(columns=["primeiro_nome_juiz", "primeiro_nome_requerente"])

print("✓ Inferência concluída!")
df_processos.head()

## 10. Salvar Dataset Final

In [ ]:
# Salvar resultado final
df_processos.to_csv("dados_processos_com_sexo.csv", index=False, encoding="utf-8")

print("✓ Dataset final salvo em 'dados_processos_com_sexo.csv'")

# Estatísticas
print(f"\n{'='*60}")
print(f"ESTATÍSTICAS FINAIS")
print(f"{'='*60}")

print(f"\nSexo dos Juízes:")
print(df_processos["sexo_juiz"].value_counts())

print(f"\nSexo dos Requerentes:")
print(df_processos["sexo_requerente"].value_counts())

print(f"\n{'='*60}")
print(f"AMOSTRA DO RESULTADO FINAL")
print(f"{'='*60}")
df_processos[["juiz", "sexo_juiz", "requerente", "sexo_requerente", "sentenca_favoravel"]].head(10)

## 11. Análise Exploratória
### Visualizar distribuição de decisões por sexo

In [ ]:
# Análise cruzada: Sexo do Juiz x Sentença
print("Distribuição de Sentenças por Sexo do Juiz:")
print(pd.crosstab(df_processos["sexo_juiz"], df_processos["sentenca_favoravel"], normalize='index') * 100)

print("\n" + "="*60)
print("\nDistribuição de Sentenças por Sexo do Requerente:")
print(pd.crosstab(df_processos["sexo_requerente"], df_processos["sentenca_favoravel"], normalize='index') * 100)